In [1]:
import pickle
import pandas as pd
import numpy as np
import re
import itertools
import json

In [2]:
data_path = "/home/ai-sjjy/workspace/juyoung/JasoseoAI_project/3.data/Keyword/"
src_path = "/home/ai-sjjy/workspace/juyoung/JasoseoAI_project/2.src/Keyword_AI/"

* 이름이 달라서 분리된것 => diff_corpName_raw
* 기업이 없는것 => noCorp_raw
* 취업TIP메뉴가 없는것 => noCorpInfo_raw
* 추출된 것 => wantNValue_raw

In [2]:
with open(data_path+'JobKorea_processed.pickle', 'rb') as f:
    jobkorea = pickle.load(f)

with open(data_path+'/Incruit/diff_corpName_raw.pickle', 'rb') as f:
    diff_corpName_raw = pickle.load(f)
    
with open(data_path+'/Incruit/noCorp_raw.pickle', 'rb') as f:
    noCorp_raw = pickle.load(f)
    
with open(data_path+'/Incruit/noCorpInfo_raw.pickle', 'rb') as f:
    noCorpInfo_raw = pickle.load(f)
    
with open(data_path+'/Incruit/wantNValue_raw.pickle', 'rb') as f:
    wantNValue_raw = pickle.load(f)

NameError: name 'data_path' is not defined

In [10]:
print(len(diff_corpName_raw))
print(len(noCorp_raw))
print(len(noCorpInfo_raw))
print(len(wantNValue_raw))

768
94
146
506


In [12]:
jobkorea.columns

Index(['company_name', 'company_url', 'apply_info', 'applicant_info', 'score',
       'score_desc', 'url', 'Question', 'content', 'comment', 'split_content'],
      dtype='object')

In [13]:
jobkorea = jobkorea[['company_name', 'content']]
jobkorea

,company_name,content
0,(주)경동나비엔,[새로운 도전을 위해 대학 시절 OO 유학을 떠나게 되었습니다. 조금 늦은 나이의 ...
1,(주)포스코인터내셔널,[어떠한 일을 시작하기 전에 두려운 마음이 들 때마다 이러한 마음가짐을 갖추고 도전...
2,㈜하나은행,"[벤처기업, 제조 공장 자동화, 금융 시스템 개발 경력저는 어려서 배운 컴퓨터 강의..."
3,조선비즈,[저는 군대에서 행정 업무를 맡으면서 문제를 해결해 임무수행에 어려움을 덜어낸 경험...
4,㈜CJ ENM,[음악을 사랑합니다. 음악뿐만 아니라 문화산업 전반에 깊은 관심이 있습니다. CJ ...
...,...,...
6626,삼화페인트공업(주),"[남들을 배려하며 착하게 사는 것이 옳다고 배우며 살아왔기 때문에, 이타적인 성향이..."
6627,LG디스플레이,"[2013년 여름, 천호식품에서 아르바이트를 했었습니다. 제품 생산에서부터 포장, ..."
6628,엘에스전선(주),[ 선배들의 추천을 받아 13년도 동아리 회장을 맡았습니다. 당시 숫기가 없어 동기...
6629,S-OIL,[ 최초를 시도한 기업은 많습니다. 하지만 최초보다 더욱 중요한 것이 최고라고 생각...


In [14]:
jobkorea['company_name'] = jobkorea['company_name'].apply(lambda x : re.sub(" +","",x.lower().replace('㈜', "").replace("㈔", "").replace("(주)", "").replace("(사)", "")))
jobkorea['company_name'] = jobkorea['company_name'].apply(lambda x : re.sub("[^가-힣a-z0-9]", "", x))

<ipython-input-14-3d12f3f8d7bd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobkorea['company_name'] = jobkorea['company_name'].apply(lambda x : re.sub(" +","",x.lower().replace('㈜', "").replace("㈔", "").replace("(주)", "").replace("(사)", "")))
<ipython-input-14-3d12f3f8d7bd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobkorea['company_name'] = jobkorea['company_name'].apply(lambda x : re.sub("[^가-힣a-z0-9]", "", x))


In [15]:
jobkorea.head(3)

,company_name,content
0,경동나비엔,[새로운 도전을 위해 대학 시절 OO 유학을 떠나게 되었습니다. 조금 늦은 나이의 ...
1,포스코인터내셔널,[어떠한 일을 시작하기 전에 두려운 마음이 들 때마다 이러한 마음가짐을 갖추고 도전...
2,하나은행,"[벤처기업, 제조 공장 자동화, 금융 시스템 개발 경력저는 어려서 배운 컴퓨터 강의..."


In [16]:
group = jobkorea.groupby('company_name')

jobkorea_group = dict()
for idx, g in group:
    sentences = []
    for sentence in g.content.tolist():
        for s in sentence:
            sentences.append(s)
    jobkorea_group[idx] = sentences

In [17]:
len(jobkorea_group)

1513

In [28]:
company_names= list(jobkorea_group.keys())

In [29]:
company_names[0]

'11번가'

In [25]:
wantNValue_raw['11']

KeyError: '11번가'

In [32]:
# 기업 - ([인재상], [핵심가치])
for k, v in wantNValue_raw.items():
    print(k, v)
    print(v[0], v[1])
    break

aj네트웍스 (['따뜻하게', '재미있게', '단순하게', '정직하게'], ['설레임', '귀기울임', '어울림'])
['따뜻하게', '재미있게', '단순하게', '정직하게'] ['설레임', '귀기울임', '어울림']


In [34]:
new_company_infos = dict()
for key, value in wantNValue_raw.items():
    if key in company_names:
        new_company_infos[key] = {"contents" : jobkorea_group[key], "인재상" : value[0], "핵심가치" : value[1]}

In [35]:
len(new_company_infos)

505

In [36]:
with open(data_path+'CompanyKeywords.json', 'w', encoding='utf-8') as f: # 회사-[질문:답변] => dict 형식
    json.dump(new_company_infos, f, indent="\t")

In [38]:
total = 0
for key, value in new_company_infos.items():
    total += len(value['contents'])

In [39]:
total

11673

## 더 크롤링

In [1]:
import requests
import re
import os
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import pickle
import selenium
from selenium import webdriver

In [2]:
data_path = "C:/Users/msi/GitHub/JasoseoAI_project/3.data/Keyword/"
src_path = "C:/Users/msi/GitHub/JasoseoAI_project/2.src/Keyword_AI/"

In [3]:
with open(data_path+'JobKorea_processed.pickle', 'rb') as f:
    jobkorea = pickle.load(f)

with open(data_path+'/Incruit/diff_corpName_raw.pickle', 'rb') as f:
    diff_corpName_raw = pickle.load(f)
    
with open(data_path+'/Incruit/noCorp_raw.pickle', 'rb') as f:
    noCorp_raw = pickle.load(f)
    
with open(data_path+'/Incruit/noCorpInfo_raw.pickle', 'rb') as f:
    noCorpInfo_raw = pickle.load(f)
    
with open(data_path+'/Incruit/wantNValue_raw.pickle', 'rb') as f:
    wantNValue_raw = pickle.load(f)

In [4]:
wanted = list(wantNValue_raw.keys())

In [5]:
wanted[0]

'aj네트웍스'

In [6]:
eng = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"]
kor = ["에이","비","씨","디","이","에프","지","에이치","아이","제이","케이","아이","엠","엔","오","피","큐","알","에스","티","유","브이","더블유","엑스","와이","제트"]

In [7]:
companys = []
for name in set(jobkorea.company_name.tolist()):
    new_name = re.sub(" +","",name.lower().replace('㈜', "").replace("㈔", "").replace("(주)", "").replace("(사)", ""))
    new_name = re.sub("[^가-힣a-z0-9]", "", new_name)
    if new_name not in wanted:
        name = name.lower().replace("㈜", "(주)")
        # [1] 영어 -> 한국어
        eng_str = ""
        for n in name:
            for idx, e in enumerate(eng): 
                if e == n: 
                    eng_str += kor[idx]
                    break
            else: eng_str += n
        # [2] 한국어 -> 영어
        kor_str = ""
        for n in name:
            for idx, k in enumerate(kor): 
                if k == n: 
                    kor_str += eng[idx]
                    break
            else: kor_str += n
        
        companys.append((name, eng_str, kor_str, new_name)) # 원래, 전처리 된 이름

In [8]:
len(companys)

1009

In [9]:
companys[0]

('(재)동그라미재단', '(재)동그라미재단', '(재)동그라미재단', '재동그라미재단')

In [10]:
for i, corps in enumerate(companys):
    print(corps[:-1][::-1])
    break

('(재)동그라미재단', '(재)동그라미재단', '(재)동그라미재단')


## 인크루트 중앙 검색창에서 가져오기

In [12]:
URL = 'https://search.incruit.com'
driver = webdriver.Chrome('C:/Users/msi/Downloads/chromedriver_win32/chromedriver.exe')
driver.implicitly_wait(3)

driver.get(url=URL)

In [15]:
def get_corp(corpName):
    enterCount = 0
    wantPeoples = []
    coreValues = []
    # /html/body/div[2]/div[1]/div[2]/form/fieldset/input
    search = driver.find_element_by_xpath("/html/body/div[2]/div[1]/div[1]/div[1]/form/fieldset/input")
    search.send_keys(corpName+'\n')
    time.sleep(0.15)
    enterCount += 1

    corpLinkText = driver.find_elements_by_xpath("/html/body/div[2]/div[2]/div[2]/div[1]/div[2]/div[1]/h3/strong/a")
    if len(corpLinkText) > 0:
        driver.get(corpLinkText[0].get_attribute("href"))
        time.sleep(0.15)
        enterCount += 1
    else:
        corpLinkText = driver.find_elements_by_xpath("/html/body/div[2]/div[1]/div[3]/div/ul/li[7]/a")
        if len(corpLinkText) > 0:
            driver.get(corpLinkText[0].get_attribute("href")) 
            time.sleep(0.15)
            enterCount += 1

            companylinks = driver.find_elements_by_class_name('compinfo')
            if len(companylinks) > 0:
                corpLinkTextList = []
                for i in range(1, 5):
                    result = driver.find_elements_by_xpath(f"/html/body/div[2]/div[2]/div[1]/div/ul/li[{i}]/div[2]/h3/a")
                    if len(result) > 0:
                        name = driver.find_elements_by_xpath(f"/html/body/div[2]/div[2]/div[1]/div/ul/li[{i}]/div[2]/h3/a/strong")
                        if len(name) > 0:
                            if len(name) < 0: 
                                corpLinkTextList = []
                                break
                            else:
                                corpLinkTextList.append([name[0].text, result[0].get_attribute("href")])
                        else:
                            corpLinkTextList.append([result[0].text, result[0].get_attribute("href")])
                    else: break
                        
                if not corpLinkTextList: 
                    print("기업이 존재하지 않습니다. : {}".format(corpName))
                    for i in range(2):
                        driver.back()
                        time.sleep(0.15)
                    return -1, corpName, None
                
                for name, links in corpLinkTextList:
                    if corpName in name:
                        driver.get(links)
                        time.sleep(0.15)
                        enterCount += 1
                        break
                else:
                    print("기업이 존재하지 않습니다. : {}".format(corpName))
                    for i in range(2):
                        driver.back()
                        time.sleep(0.15)
                    return -1, corpName, None
            else:
                print("기업이 존재하지 않습니다. : {}".format(corpName))
                for i in range(2):
                    driver.back()
                    time.sleep(0.15)
                return -1, corpName, None
            
        else:
            print("기업이 존재하지 않습니다. : {}".format(corpName))
            driver.back()
            return -1, corpName, None
    
    # 회사 페이지에서 메뉴텝에서 취업TIP이 있나 확인
    menuTap = driver.find_element_by_class_name('tabmenu_temp')
    menuList = menuTap.find_elements_by_tag_name('li')
    menuTextList = [menu.get_attribute('textContent').strip() for menu in menuTap.find_elements_by_tag_name('span')]

    if '취업TIP' in menuTextList:
        idx = menuTextList.index('취업TIP')
        menuList[idx].find_element_by_tag_name('a').send_keys('\n')
        time.sleep(0.15)
        enterCount += 1
    else:
        print(corpName, ": 취업TIP not exist")
        for i in range(enterCount):
            driver.back()
        return -2, corpName, None
    
    # 취업TIP에서 인재상 찾기 또는 핵심가치 찾기
    contents = driver.find_elements_by_class_name('contPre')
    for i, c in enumerate(contents):
        # 표로 되어 있는 경우
        try:
            rightpeopleClass = c.find_element_by_class_name('gongche_con_rightpeople')
            rightpeopletdTags = rightpeopleClass.find_elements_by_tag_name('td')
            for i in rightpeopletdTags:
                wantPeoples.append(i.get_attribute('textContent'))
        except:
            pass
        try:
            corvalueClass = c.find_element_by_class_name('gongche_con_corevalue')
            corvaluetdTags = corvalueClass.find_elements_by_tag_name('td')
            for i in corvaluetdTags:
                coreValues.append(i.get_attribute('textContent'))
        except:
            headName = c.find_elements_by_tag_name('h3')
            headName = [name.get_attribute('textContent').strip() for name in headName]
            if not(('인재상' in headName) or ('핵심가치' in headName)):
                continue

            headContent = c.find_elements_by_class_name('contPresent')
            headContent = [name.get_attribute('textContent').strip() for name in headContent]

            if '인재상' in headName:
                idx = headName.index('인재상')
                wantPeoples = headContent[idx].strip().split('/')
                wantPeoples = [wantPeople.strip() for wantPeople in wantPeoples]
            else:
                print(corpName, '인재상 not exist')

            if '핵심가치' in headName:
                idx = headName.index('핵심가치')
                coreValues = headContent[idx].strip().split('/')
                coreValues = [coreValue.strip() for coreValue in coreValues]
            else:
                print(corpName, '핵심가치 not exist')
    for i in range(enterCount):
        driver.back()
    
    return corpName, wantPeoples, coreValues

In [16]:
noCorp = []
diff_corpName = []
noCorpInfo = []
wantNValue = {}

for i, corps in enumerate(companys):
    print(i+1,' ',end='')
    try:
        company_com = set(corps[:-1][::-1])
        for corp in company_com:
            state, wantPeoples, coreValues = get_corp(corp)
            # 정상 출력
            if state == corp:
                wantNValue[corp] = (wantPeoples, coreValues)
                print("success get data: ", corp)
                break
        else:
            # 이름이 다른 경우
            if state == -1:
                diff_corpName.append((wantPeoples, coreValues))
            # 취업TIP이 없는 경우
            elif state == -2:
                noCorpInfo.append(wantPeoples)
                
    except Exception as e:
        noCorp.append(corp)
        print("{}: {}".format(corp, e))
        driver.get('https://search.incruit.com')
        time.sleep(0.2)

1  기업이 존재하지 않습니다. : (재)동그라미재단
2  기업이 존재하지 않습니다. : (주)현대에e치cn
success get data:  (주)현대에이치씨엔
3  success get data:  인천종합에너지(주)
4  기업이 존재하지 않습니다. : 경기농식품유통진흥원
기업이 존재하지 않습니다. : 경기농식품u통진흥원
5  세아제강 핵심가치 not exist
success get data:  세아제강
6  (주)알티캐스트 : 취업TIP not exist
기업이 존재하지 않습니다. : (주)rt캐스트
7  success get data:  (주)카스
8  success get data:  미래에셋생명보험(주)
9  success get data:  (주)한솔홈데코
10  기업이 존재하지 않습니다. : (주)롯데닷컴
11  한국방송광고진흥공사 핵심가치 not exist
success get data:  한국방송광고진흥공사
12  success get data:  (주)넥센
13  넥슨지티(주) : 취업TIP not exist
기업이 존재하지 않습니다. : 넥슨gt(주)
14  success get data:  인천글로벌캠퍼스운영
15  기업이 존재하지 않습니다. : 동양p스톤(주)
success get data:  동양피스톤(주)
16  (주)에스아이티 : 취업TIP not exist
기업이 존재하지 않습니다. : (주)에스아et
17  한양대학교 : 취업TIP not exist
18  success get data:  도시바일렉트로닉스코리아(주)
19  success get data:  (주)kcc
20  success get data:  (주)한빛소프트
21  기업이 존재하지 않습니다. : p에e치에e에차u한회사
기업이 존재하지 않습니다. : 피에이치에이에차유한회사
22  success get data:  (주)유비쿼스
23  한국교육방송공사 핵심가치 not exist
success get data:  한국교육방송공사
24  인천교통공사 인재상 not 

In [17]:
import pickle

f = open("noCorp_raw.pickle", "wb")
pickle.dump(noCorp, f)
f.close()

f = open("diff_corpName_raw.pickle", "wb")
pickle.dump(diff_corpName, f)
f.close()

f = open("noCorpInfo_raw.pickle", "wb")
pickle.dump(noCorpInfo, f)
f.close()

f = open("wantNValue_raw.pickle", "wb")
pickle.dump(wantNValue, f)
f.close()

In [18]:
len(wantNValue)

420

In [19]:
for key, value in wantNValue.items():
    print(value)
    break

(['01윤리인', '02의지인', '03조직인'], [])
